### Delta Live Tables Using SQL

> #### Note: Use `Sales-Data-Generator` notebook to add the data and query the result
- Observe the code in this notebook
- Open the notebook and run **cell number 1 to 7** only.
- Create a DLT pipeline and provide the following in formation:
  - Pipeline name: `<Enter unique name>`
  - Product edition: **Advance**
  - Pipeline mode: **Conitnues**
  - Paths: `<Navigate to this notebook and select it>`.
  - Storage option: **hive_metastore**
  - Storage location: `<your-mount-point-name/tables>`
  - Default schema: **dltdb**
  - Cluster mode: **Fixed size**
  - Workers: **0**
  - Advance Configuration:
    - Key: **path**
    - Value: `dbfs:/<your-mount-point-name/sales_order>`
  - Driver type: **Ds3v2**
  - Click Create
  - On the DLT pipeline page, click Start.
  - Once Started, go back to `Sales-Data-Generator` and run cell no 8 to view the output.
  - Now run cell no 7 again to load more and run cell 8 again.

### Create Bronze Layer with Auto Loader

In [0]:
CREATE OR REFRESH STREAMING TABLE bronze_sales (
  CONSTRAINT positive_quantity EXPECT (quantity > 0) ON VIOLATION DROP ROW
)
AS
SELECT * FROM cloud_files(
  "${path}",
  "json",
  map(
    "cloudFiles.inferColumnTypes", "true"
  )
);

### Create a materialized view for product lookups (to ensure freshness)

In [0]:
CREATE OR REPLACE MATERIALIZED VIEW mv_product_lookup AS
SELECT * FROM hive_metastore.dltdb.products;

### Create Silver streaming table with product enrichment and expectation

#### Understanding the LIVE Keyword and STREAM Method in DLT
- The **LIVE** keyword is a namespace used to refer to any DLT-managed object (table, view) within the same pipeline.

- The **STREAM(...)** method is used to read a table as a streaming source, enabling incremental (append-only) data processing.

- When using STREAM(...) to read from a source:

  - The target table must be defined as a streaming table using CREATE OR REFRESH STREAMING TABLE.
  - This ensures incremental ingestion and avoids full refresh behavior.

In [0]:
CREATE OR REFRESH STREAMING TABLE silver_sales AS
SELECT
  b.order_id,
  b.customer_id,
  b.product_id,
  p.product_name,
  p.unit_price,
  b.quantity,
  (b.quantity * p.unit_price) AS total_price,
  b.order_timestamp
FROM STREAM(LIVE.bronze_sales) AS b
JOIN LIVE.mv_product_lookup AS p
  ON b.product_id = p.product_id;

### Gold – Business Aggregation

In [0]:
CREATE OR REPLACE MATERIALIZED VIEW gold_top_products AS
SELECT
  product_id,
  product_name,
  SUM(quantity) AS total_quantity_sold,
  SUM(total_price) AS total_revenue
FROM LIVE.silver_sales
GROUP BY product_id, product_name
ORDER BY total_quantity_sold DESC;